# 목차
<hr/>

<ol>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">필요한 모듈 불러오기</i>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 데이터, 평가 기준 살펴보기</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> 대회 소개, 평가 기준 살펴보기
            </li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> 데이터 살펴보기</li>
        </ul>
    </li><li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 2</span> : Baseline 셋팅하기</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> 데이터 불러오기
            </li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> date 칼럼 정리하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">3. </span> id 삭제하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">4. </span> 학습 데이터에서 target값 분리하기</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">5. </span> train, test데이터 분리하기</li>
        </ul>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : test.csv에 실제로 적용해보기</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> test 데이터 준비하기
            </li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> 데이터 이해</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">3. </span> train, test 데이터 분리</li>
        </ul>
    </li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : 모델 학습하기</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> test 데이터 준비하기
            </li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> 데이터 이해</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">3. </span> train, test 데이터 분리</li>
        </ul>
    </li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;"><span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : test.csv에 실제로 적용해보기</i>
        <ul style = "margin-top:10px; list-style:none;">
            <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">1. </span> test 데이터 준비하기
            </li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">2. </span> 데이터 이해</li>
            <li style = "margin-bottom:5px;"><span style = "font-weight:bold;color:#0172d4;">3. </span> train, test 데이터 분리</li>
        </ul>
    </li>
    <li style = "margin-bottom:30px; list-style: none;"><i style = "font-weight:bold;font-style: normal;">회고</i>
</ol>

# 필요한 모듈 불러오기

In [1]:
import warnings
warnings.filterwarnings("ignore")

import lightgbm          as lgb
import matplotlib.pyplot as plt
import missingno         as msno
import numpy             as np
import os
import pandas            as pd
import seaborn           as sns
import xgboost           as xgb


from os.path                 import join
from sklearn.ensemble        import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_validate, cross_val_score, train_test_split
from sklearn.metrics         import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV


from catboost import CatBoostRegressor

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 1</span> : 데이터, 평가 기준 살펴보기

[참여할 캐글 대회](https://www.kaggle.com/c/2019-2nd-ml-month-with-kakr)

## 1. 대회 소개, 평가 기준 살펴보기
<hr/>

### (1)Description, 대회 소개

<p style= "font-size:15px; line-height:24px;">데이터는 20개의 변수를 가지고 있으며, 어떤 조건을 가진 집의 가격이 높고 낮은지를 예측하는 모델을 만드는 것이 목표이다.<br/>
따라서, 이 문제는 집값을 예측하는 <span style = 'background-color:#fff5b1; padding:0.2px;'>회귀</span> 문제이다.</p>

<div style = "height: 25px;"></div>

### (2) Evaluation, 점수 평가 기준
<p style= "font-size:15px; line-height:24px;">
평가 방식은 <span style = 'background-color:#fff5b1; padding:0.2px;'>RMSE (Root Mean Squared Error)</span> 이다.
$${\sqrt{ {1 \over N} \sum{(yt - y{pr})}^2}} $$</p>

<div style = "height: 50px;"><div/>

## 2. 데이터 살펴보기
<hr/>

### (1) 데이터 설명

<ul>
	<li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">train.csv</span> - 예측 모델을 만들기 위해 사용하는 학습 데이터이다. 집의 정보와 예측할 변수인 가격(Price) 변수를 가지고 있다.</li>
	<li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">test.csv</span> - 학습셋으로 만든 모델을 가지고 예측할 가격(Price) 변수를 제외한 집의 정보가 담긴 테스트 데이터이다.</li>
	<li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">sample_submission.csv</span> - 제출시 사용할 수 있는 예시 submission.csv 파일</li>
</ul>

#### Data fields

<ul>
	<li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">ID : </span> 집을 구분하는 번호</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">date : </span> 집을 구매한 날짜</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">price : </span> 집의 가격(Target variable)</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">bedrooms : </span> 침실의 수</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">bathrooms : </span> 화장실의 수</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">sqft_living : </span> 주거 공간의 평방 피트(면적)</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">sqft_lot : </span> 부지의 평방 피트(면적)</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">floors : </span> 집의 층 수</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">waterfront : </span> 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">view : </span> 집이 얼마나 좋아 보이는지의 정도</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">condition : </span> 집의 전반적인 상태</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">grade : </span> King County grading 시스템 기준으로 매긴 집의 등급</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">sqft_above : </span> 지하실을 제외한 평방 피트(면적)</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">sqft_basement : </span> 지하실의 평방 피트(면적)</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">yr_built : </span> 지어진 년도</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">yr_renovated : </span> 집을 재건축한 년도</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">zipcode : </span> 우편번호</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">lat : </span> 위도</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">long : </span> 경도</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">sqft_living15 : </span> 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)</li>
    <li style = "margin-bottom:3px;"><span style = "font-weight:bold;color:#0172d4;">sqft_lot15 : </span> 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)</li>
</ul>

<p style= "font-size:15px; line-height:24px;">우리가 예측해야 하는 컬럼은 price(집의 가격)이다. <br/>따라서 <span style = 'background-color:#fff5b1; padding:0.2px;'>타겟값은 'price'이다.</span></p>

<div style = "height: 25px;"><div/>

### (2) 목표

<p style= "font-size:15px; line-height:24px;">train.csv를 활용해서 데이터를 전처리하여 모델을 학습시킨 후, <br/>test.csv 파일의 데이터에 대해 price를 예측한 후 sample_submission.csv의 형식에 맞는 형태로 캐글에 제출을 할 것이다.</p>

<div style = "height: 50px;"><div/>

# <span style = "font-weight:bold;color:#0172d4;">STEP 2</span> : Baseline 셋팅하기

## 1. 데이터 불러오기
<hr/>

In [5]:
train = pd.read_csv('data_2/train.csv')
test = pd.read_csv('data_2/test.csv')

In [6]:
train

id             date      price  bedrooms  bathrooms  sqft_living  \
0          0  20141013T000000   221900.0         3       1.00         1180   
1          1  20150225T000000   180000.0         2       1.00          770   
2          2  20150218T000000   510000.0         3       2.00         1680   
3          3  20140627T000000   257500.0         3       2.25         1715   
4          4  20150115T000000   291850.0         3       1.50         1060   
...      ...              ...        ...       ...        ...          ...   
15030  15030  20141014T000000   610685.0         4       2.50         2520   
15031  15031  20150326T000000  1007500.0         4       3.50         3510   
15032  15032  20140521T000000   360000.0         3       2.50         1530   
15033  15033  20150223T000000   400000.0         4       2.50         2310   
15034  15034  20141015T000000   325000.0         2       0.75         1020   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
0          5650     1.0           0     0  ...      7        1180   
1         10000     1.0           0     0  ...      6         770   
2          8080     1.0           0     0  ...      8        1680   
3          6819     2.0           0     0  ...      7        1715   
4          9711     1.0           0     0  ...      7        1060   
...         ...     ...         ...   ...  ...    ...         ...   
15030      6023     2.0           0     0  ...      9        2520   
15031      7200     2.0           0     0  ...      9        2600   
15032      1131     3.0           0     0  ...      8        1530   
15033      5813     2.0           0     0  ...      8        2310   
15034      1076     2.0           0     0  ...      7        1020   

       sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
0                  0      1955             0    98178  47.5112 -122.257   
1                  0      1933             0    98028  47.7379 -122.233   
2                  0      1987             0    98074  47.6168 -122.045   
3                  0      1995             0    98003  47.3097 -122.327   
4                  0      1963             0    98198  47.4095 -122.315   
...              ...       ...           ...      ...      ...      ...   
15030              0      2014             0    98056  47.5137 -122.167   
15031            910      2009             0    98136  47.5537 -122.398   
15032              0      2009             0    98103  47.6993 -122.346   
15033              0      2014             0    98146  47.5107 -122.362   
15034              0      2008             0    98144  47.5941 -122.299   

       sqft_living15  sqft_lot15  
0               1340        5650  
1               2720        8062  
2               1800        7503  
3               2238        6819  
4               1650        9711  
...              ...         ...  
15030           2520        6023  
15031           2050        6200  
15032           1530        1509  
15033           1830        7200  
15034           1020        1357  

[15035 rows x 21 columns]

<div style = "height: 50px;"></div>

## 2. date 칼럼 정리하기
<hr/>

<p style= "font-size:15px; line-height:24px;">20141013T000000 형식의 데이터를 연/월 데이터만 사용하기 위해 <span style = 'background-color:#fff5b1; padding:0.2px;'>str(x[:6]) 으로 연/월까지 자른다.</span></p>

In [7]:
# date 칼럼 정리
train['date'] = train['date'].apply(lambda x : str(x[:6])).astype(int)
train.head()

id    date     price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  \
0   0  201410  221900.0         3       1.00         1180      5650     1.0   
1   1  201502  180000.0         2       1.00          770     10000     1.0   
2   2  201502  510000.0         3       2.00         1680      8080     1.0   
3   3  201406  257500.0         3       2.25         1715      6819     2.0   
4   4  201501  291850.0         3       1.50         1060      9711     1.0   

   waterfront  view  ...  grade  sqft_above  sqft_basement  yr_built  \
0           0     0  ...      7        1180              0      1955   
1           0     0  ...      6         770              0      1933   
2           0     0  ...      8        1680              0      1987   
3           0     0  ...      7        1715              0      1995   
4           0     0  ...      7        1060              0      1963   

   yr_renovated  zipcode      lat     long  sqft_living15  sqft_lot15  
0             0    98178  47.5112 -122.257           1340        5650  
1             0    98028  47.7379 -122.233           2720        8062  
2             0    98074  47.6168 -122.045           1800        7503  
3             0    98003  47.3097 -122.327           2238        6819  
4             0    98198  47.4095 -122.315           1650        9711  

[5 rows x 21 columns]

<div style = "height: 50px;"></div>

## 3. id 삭제하기
<hr/>

In [8]:
# id 삭제
del train['id']
print(train.columns)

Index(['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


<div style = "height: 50px;"></div>

## 4. 학습 데이터에서 target값 분리하기
<hr/>

### 삭제 전, 타겟값 y에 저장하기

In [9]:
y = train['price']

<div style = "height: 25px;"></div>

### 학습 데이터에서 target(price)값 삭제

In [10]:
del train['price']

<div style = "height: 25px;"></div>

### x값 지정하기

In [11]:
x = train

<div style = "height: 50px;"></div>

## 5. train, test데이터 분리하기
<hr/>

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

<div style = "height: 50px;"></div>

# <span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : 모델 개요

## 1. 사용한 모델
<hr/>
<p style= "font-size:20px;">CatBoostRegressor</p>

<div style = "height: 50px;"></div>

## 2. CatBoostRegressor란?
<hr/>

<p style= "font-size:15px; line-height:24px;">GBM(Gradient Boosting Machines) 알고리즘을 기반으로, <span style = 'background-color:#fff5b1; padding:0.2px;'>GBM의 단점을 개선하여 만들어진 알고리즘이다.</span></p>

- <span style = 'background-color:#fff5b1; padding:0.2px;'>과적합(overfitting)</span> 문제를 해결
- GBM 계열의 알고리즘인 XGBoost, LightGBM보다 <span style = 'background-color:#fff5b1; padding:0.2px;'>학습속도가 빠르다.</span>
- XGBoost, LightGBM이 하이퍼 파라미터에 따라 성능이 달라지는 문제가 있으나,<br/> 캣부스트는 이러한 문제를 해소하였다. 

### GBM(Gradient Boosting Machines)이란?

- 주어진 데이터에 대해 계속적으로 학습(fit)하여 모델을 만든 후, 다른 데이터를 예측(predict)함으로서, 간차를 줄여나가는 방식이다.
- 최초의 데이터 이후는 계산한 잔차에 포커스를 맞추어 학습하므로, 새로운 데이터에 대한 예측이 떨어질 수 있다. (overfitting을 유발할 가능성이 높다)

### 비교하기

<table style="float:left; font-size:15px; width:80%; border:0.2px solid">
    <th style = "text-align:center; width:33.33%; border:0.2px solid">LightGBM</th>
    <th style = "text-align:center; width:33.33%; border:0.2px solid">XGBoost</th>
    <th style = "text-align:center; width:33.33%; border:0.2px solid">Catboost</th>
    <tr>       
        <td style = "text-align:center; width:33.33%; border:0.2px solid">DFS(깊이 우선 탐색)<br/> 트리를 먼저 깊게 형성한다.</td>
        <td style = "text-align:center; width:33.33%; border:0.2px solid">BFS(너비 우선 탐색)<br/> 트리를 먼저 넓게 형성한다.</td>
        <td style = "text-align:center; width:33.33%; border:0.2px solid">특성(feature)를 모두 동일하게<br/> 대칭적으로 트리를 형성하여<br/><span style = 'background-color:#fff5b1; padding:0.2px;'>예측 시간을 감소</span>시켰다.</td>
    </tr>	
</table>

<div style = "height: 50px;"></div>

## 3. 모델을 선택한 이유?
<hr/>

<p style= "font-size:15px; line-height:26px;">LightGBM와 XGBoost에 대해 알아보던 중, CatBoost를 같이 비교하는 글을 많이 접했습니다.<br/>
특히 XGBoost와 CatBoost는 같이 나오는 경우가 많아 읽어보았는데, LightGBM와 XGBoost는 모두 GBM 알고리즘 기반이며, 캣부스트는 GBM의 단점을 개선하여 만들어진 알고리즘이라 하여 사용하게 되었습니다.<br/><br/>
또한, LightGBM과 XGBoost는 하이퍼 파라미터에 민감하므로, 최적의 파라미터를 찾아야 성능이 좋지만, 캣부스트는 하이퍼 파라미터를 설정하지 않아도 모델의 성능이 우수하며, 하이퍼 파라미터를 설정하면 더욱 좋은 성능을 낼 수 있다는 것에 매력을 느꼈습니다.<br/><br/>
처음에는 캣부스트가 결측치가 많은 데이터에는 부적합한 모델이라 하여 걱정했지만, 캐글 데이터를 확인해본 결과, 결측치가 없어 모델이 잘 작동할 것이라 예측합니다.</p>

<div style = "height: 50px;"></div>

## 2. 모델 학습하기
<hr/>

In [9]:
moedel_cat = CatBoostRegressor(random_state=42)

In [10]:
score_cat = moedel_cat.fit(x_train, y_train)

Learning rate set to 0.059386
0:	learn: 365954.0982985	total: 49.1ms	remaining: 49.1s
1:	learn: 352886.1971015	total: 51.5ms	remaining: 25.7s
2:	learn: 341049.9718505	total: 53.8ms	remaining: 17.9s
3:	learn: 328746.8191836	total: 55.8ms	remaining: 13.9s
4:	learn: 317463.3161305	total: 57.7ms	remaining: 11.5s
5:	learn: 308097.2028307	total: 59.4ms	remaining: 9.83s
6:	learn: 298370.9355166	total: 61ms	remaining: 8.65s
7:	learn: 289586.9395552	total: 62.5ms	remaining: 7.75s
8:	learn: 280596.8517143	total: 64.2ms	remaining: 7.07s
9:	learn: 272359.1328115	total: 65.8ms	remaining: 6.52s
10:	learn: 264924.5986983	total: 67.3ms	remaining: 6.05s
11:	learn: 258018.3683335	total: 68.8ms	remaining: 5.67s
12:	learn: 251206.4232732	total: 70.2ms	remaining: 5.33s
13:	learn: 244703.5122395	total: 72ms	remaining: 5.07s
14:	learn: 238160.9894305	total: 73.5ms	remaining: 4.82s
15:	learn: 232721.6865524	total: 74.8ms	remaining: 4.6s
16:	learn: 228178.3120848	total: 76.3ms	remaining: 4.41s
17:	learn: 22334

201:	learn: 98405.2654553	total: 350ms	remaining: 1.38s
202:	learn: 98243.1470377	total: 352ms	remaining: 1.38s
203:	learn: 98115.0753551	total: 354ms	remaining: 1.38s
204:	learn: 97992.4381253	total: 355ms	remaining: 1.38s
205:	learn: 97914.1437360	total: 358ms	remaining: 1.38s
206:	learn: 97825.7916577	total: 359ms	remaining: 1.38s
207:	learn: 97717.8344669	total: 361ms	remaining: 1.37s
208:	learn: 97623.1516135	total: 362ms	remaining: 1.37s
209:	learn: 97441.8342778	total: 364ms	remaining: 1.37s
210:	learn: 97324.7126114	total: 366ms	remaining: 1.37s
211:	learn: 97191.2273205	total: 368ms	remaining: 1.37s
212:	learn: 97048.6146516	total: 369ms	remaining: 1.36s
213:	learn: 96934.8365991	total: 373ms	remaining: 1.37s
214:	learn: 96808.3502872	total: 374ms	remaining: 1.37s
215:	learn: 96693.1933201	total: 376ms	remaining: 1.36s
216:	learn: 96546.1320622	total: 377ms	remaining: 1.36s
217:	learn: 96409.1630033	total: 379ms	remaining: 1.36s
218:	learn: 96299.4908246	total: 381ms	remaining

446:	learn: 78651.4923791	total: 734ms	remaining: 908ms
447:	learn: 78580.8186828	total: 736ms	remaining: 907ms
448:	learn: 78526.9690922	total: 737ms	remaining: 905ms
449:	learn: 78489.4501094	total: 739ms	remaining: 903ms
450:	learn: 78419.0887416	total: 740ms	remaining: 901ms
451:	learn: 78380.8811757	total: 743ms	remaining: 900ms
452:	learn: 78315.6762603	total: 744ms	remaining: 899ms
453:	learn: 78245.7896075	total: 746ms	remaining: 897ms
454:	learn: 78196.2221698	total: 747ms	remaining: 895ms
455:	learn: 78176.5184058	total: 749ms	remaining: 894ms
456:	learn: 78139.0199825	total: 751ms	remaining: 892ms
457:	learn: 78078.2991797	total: 753ms	remaining: 891ms
458:	learn: 78050.6650736	total: 755ms	remaining: 890ms
459:	learn: 77992.1059205	total: 757ms	remaining: 888ms
460:	learn: 77963.9204319	total: 758ms	remaining: 886ms
461:	learn: 77924.7694648	total: 759ms	remaining: 884ms
462:	learn: 77884.7738806	total: 761ms	remaining: 882ms
463:	learn: 77848.0852810	total: 762ms	remaining

687:	learn: 69119.8064471	total: 1.12s	remaining: 508ms
688:	learn: 69085.1103475	total: 1.12s	remaining: 506ms
689:	learn: 69061.2571379	total: 1.12s	remaining: 505ms
690:	learn: 69027.0405659	total: 1.12s	remaining: 503ms
691:	learn: 69005.9326040	total: 1.13s	remaining: 501ms
692:	learn: 68983.7910124	total: 1.13s	remaining: 500ms
693:	learn: 68952.0255079	total: 1.13s	remaining: 498ms
694:	learn: 68927.7335026	total: 1.13s	remaining: 497ms
695:	learn: 68896.0650252	total: 1.13s	remaining: 495ms
696:	learn: 68852.2103216	total: 1.14s	remaining: 494ms
697:	learn: 68818.7672910	total: 1.14s	remaining: 492ms
698:	learn: 68791.8867643	total: 1.14s	remaining: 490ms
699:	learn: 68757.0631077	total: 1.14s	remaining: 489ms
700:	learn: 68708.5976617	total: 1.14s	remaining: 487ms
701:	learn: 68704.9384402	total: 1.14s	remaining: 485ms
702:	learn: 68674.6619077	total: 1.15s	remaining: 484ms
703:	learn: 68644.6290600	total: 1.15s	remaining: 482ms
704:	learn: 68603.4385099	total: 1.15s	remaining

927:	learn: 62886.6941066	total: 1.5s	remaining: 117ms
928:	learn: 62858.4146032	total: 1.51s	remaining: 115ms
929:	learn: 62839.6476788	total: 1.51s	remaining: 114ms
930:	learn: 62799.5173858	total: 1.51s	remaining: 112ms
931:	learn: 62760.5588287	total: 1.51s	remaining: 110ms
932:	learn: 62738.3629583	total: 1.51s	remaining: 109ms
933:	learn: 62722.4657207	total: 1.51s	remaining: 107ms
934:	learn: 62692.6721597	total: 1.51s	remaining: 105ms
935:	learn: 62657.4096544	total: 1.52s	remaining: 104ms
936:	learn: 62648.5400706	total: 1.52s	remaining: 102ms
937:	learn: 62627.4641066	total: 1.52s	remaining: 101ms
938:	learn: 62607.9074390	total: 1.52s	remaining: 98.9ms
939:	learn: 62582.5378312	total: 1.52s	remaining: 97.3ms
940:	learn: 62564.3747603	total: 1.53s	remaining: 95.7ms
941:	learn: 62550.8618497	total: 1.53s	remaining: 94.1ms
942:	learn: 62505.7566952	total: 1.53s	remaining: 92.4ms
943:	learn: 62473.2212945	total: 1.53s	remaining: 90.8ms
944:	learn: 62448.9796415	total: 1.53s	rema

In [11]:
score_cat.feature_importances_

array([ 0.7414587 ,  0.22360852,  1.73560864, 17.67106934,  1.62465955,
        0.36386479,  2.9080316 ,  2.57048232,  0.62530297, 15.61109903,
        4.17191655,  1.02976815,  3.0685218 ,  0.73021257,  2.1958995 ,
       26.53232869, 12.54121174,  5.03245762,  0.62249792])

In [12]:
score_cat.best_score_

{'learn': {'RMSE': 61309.90095563938}}

In [13]:
cat_pred = score_cat.predict(x_test)

In [14]:
pd.DataFrame(cat_pred).isnull().sum()

0    0
dtype: int64

<div style = "height: 50px;"></div>

# <span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : test.csv에 실제로 적용해보기

## 1.  test 데이터 준비하기
<hr/>

### id 삭제하기

In [15]:
sub_id = test['id'] 
del test['id']
print(test.columns)

Index(['date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


<div style = "height: 25px;"></div>

### date 칼럼 정리하기

In [16]:
test['date'] = test['date'].apply(lambda i : i[:6]).astype(int)
test.head()

date  bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  \
0  201412         3       2.25         2570      7242     2.0           0   
1  201412         4       3.00         1960      5000     1.0           0   
2  201405         4       4.50         5420    101930     1.0           0   
3  201504         3       1.00         1780      7470     1.0           0   
4  201503         3       2.50         1890      6560     2.0           0   

   view  condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0     0          3      7        2170            400      1951          1991   
1     0          5      7        1050            910      1965             0   
2     0          3     11        3890           1530      2001             0   
3     0          3      7        1050            730      1960             0   
4     0          3      7        1890              0      2003             0   

   zipcode      lat     long  sqft_living15  sqft_lot15  
0    98125  47.7210 -122.319           1690        7639  
1    98136  47.5208 -122.393           1360        5000  
2    98053  47.6561 -122.005           4760      101930  
3    98146  47.5123 -122.337           1780        8113  
4    98038  47.3684 -122.031           2390        7570

<div style = "height: 50px;"></div>

## 2. test 데이터 예측하기
<hr/>

In [15]:
cat_pred = score_cat.predict(test)

NameError: name 'score_cat' is not defined

<div style = "height: 50px;"></div>

# <span style = "font-weight:bold;color:#0172d4;">STEP 5</span> : 제출하기

## 1. submission 파일 만들기
<hr/>

In [18]:
submission_path = join('data/sample_submission.csv')
submission = pd.read_csv(submission_path)
submission.head()

id   price
0  15035  100000
1  15036  100000
2  15037  100000
3  15038  100000
4  15039  100000

In [19]:
submission['price'] = cat_pred
submission.head()

id         price
0  15035  5.165286e+05
1  15036  4.777484e+05
2  15037  1.434916e+06
3  15038  2.945613e+05
4  15039  3.225232e+05

In [20]:
submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format('data_2', 'cat', '0.164399')
submission.to_csv(submission_csv_path, index=False)
print(submission_csv_path)

data_2/submission_cat_RMSLE_0.164399.csv


<div style = "height: 50px;"></div>

## 2. 캐글에 제출하기
<hr/>

## 랜덤 그리드 서치 적용하여 캣 부스트 하이퍼 파라메터 설정

In [ ]:
CatBoostRegressor

In [21]:
parmams = {'learning_rate' : uniform(0.0001, 0.01),
          'iterations' : randint(500, 2000),
          'depth' : 7,
          }

In [22]:
gs = RandomizedSearchCV(CatBoostRegressor(random_state = 42), n_iter = 100, n_jobs = -1)

TypeError: __init__() missing 1 required positional argument: 'param_distributions'